# NLP - Malayalam

In [ ]:
%%capture
!pip install -qq transformers transformers['sentencepiece'] torch torchaudio pydub soundfile datasets jiwer wandb

In [ ]:
import transformers
import torchaudio
import torch
import IPython
from pydub import AudioSegment
import ipywidgets as widgets
from transformers import  Wav2Vec2Processor,Wav2Vec2ForCTC

In [ ]:
uploader = widgets.FileUpload(description="Audio Upload", accept='.wav', multiple=False)
uploader

In [ ]:
with open(list(uploader.value.keys())[0], "w+b") as i:
    i.write(uploader.data[0])

In [ ]:
audio_filename = uploader.data[0]

In [ ]:
import soundfile as sf
data, samplerate = sf.read(audio_filename)
data, samplerate
# sf.write('newSpeech.wav', uploader.data[0], 16000)
IPython.display.Audio(audio_filename)

## Malayalam - Speech to Text Prediction

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("gvs/wav2vec2-large-xlsr-malayalam")
model = Wav2Vec2ForCTC.from_pretrained("gvs/wav2vec2-large-xlsr-malayalam")

In [ ]:
batch = {
         "path":'/content/' + audio_filename
        }

resampler = torchaudio.transforms.Resample(48_000, 16_000)

def speech_file_to_array_fn(batch):
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch

input_voice = speech_file_to_array_fn(batch)["speech"]#[:2]

In [ ]:
inputs = processor(input_voice, sampling_rate=16_000, return_tensors="pt", padding=True)

In [ ]:
with torch.no_grad():
  logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits
  #logits = model(torch.squeeze(inputs.input_values), attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
#print("Reference:", test_dataset["sentence"])

# References:

* Fine-tuning XLSR-Wav2Vec2 for Multi-Lingual ASR with 🤗 Transformers

  https://github.com/gauthamsuresh09/wav2vec2-large-xlsr-53-malayalam/blob/main/fine-tune-xlsr-wav2vec2-on-malayalam-asr-with-transformers_v2.ipynb


* gvs/wav2vec2-large-xlsr-malayalam

  https://huggingface.co/gvs/wav2vec2-large-xlsr-malayalam
  
  https://github.com/gauthamsuresh09/wav2vec2-large-xlsr-53-malayalam/blob/main/make_hf_dataset.ipynb
  
  https://github.com/gauthamsuresh09/wav2vec2-large-xlsr-53-malayalam/blob/main/fine-tune-xlsr-wav2vec2-on-malayalam-asr-with-transformers_v2.ipynb

* Kaggle Dataset

  https://www.kaggle.com/kavyamanohar/indic-tts-malayalam-speech-corpus

* Ramayanam

  https://www.youtube.com/playlist?list=PLSU-mNMlRpjS6uDTWFyhMXbKm-_JXi14W

* Audio/Video

  https://nbviewer.jupyter.org/github/rajeshradhakrishnanmvk/kitchenpy/blob/master/00_core.ipynb

* https://huggingface.co/gvs/wav2vec2-large-xlsr-malayalam

* Rachel Thomas (NLP Course)

  https://www.youtube.com/watch?v=tG3pUwmGjsc&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=2

* Contradictory, My Dear Watson
  https://www.kaggle.com/rajeshradhakrishnan/raj-contradictory-my-dear-watson

* fastai, Huggingface & Weights and Bias
  
  [Lecture - 1 Colab](https://colab.research.google.com/gist/ohmeow/d5a35fc237fd416ddba3ac82dca38bd5/fastai-huggingface-session-1-transformers-what-can-they-do.ipynb)
  
  [Lecture - 2 Colab](https://colab.research.google.com/gist/ohmeow/db9071ecf96e867a6f67349162ad1ab6/fastai-huggingface-session-2-using-transformers.ipynb)

* Language Model Zoo

    https://forums.fast.ai/t/language-model-zoo-gorilla/14623
    
    https://github.com/goru001/nlp-for-malyalam
    
    https://github.com/qburst/common-crawl-malayalam
    
    https://forums.fast.ai/t/language-models-for-multiple-languages/14517/10

    https://github.com/google/sentencepiece

    https://github.com/goru001/nlp-for-malyalam/blob/master/language-model/Malyalam_Language_Model_ULMFiT.ipynb

    https://github.com/goru001/inltk

* Books

  [Natural Language Processing with Python](https://www.nltk.org/book/)

  [Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/information-retrieval-book.html)

* Productionize 

  [Dockerize Hugging Face](https://github.com/AbinayaM02/HFServing)

  [Heroku api] (https://huggingface-raj.herokuapp.com/api/get-classify)
